In [1]:
from sklearn.tree import DecisionTreeClassifier
import pandas as pd
import sklearn
import scipy
import numpy as np
import matplotlib.pyplot as plt
from html_sanitizer import Sanitizer
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GroupKFold
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
import random as rnd
from sklearn.metrics import f1_score
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from pymystem3 import Mystem
from string import punctuation
from nltk.stem import WordNetLemmatizer
import bs4
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from sklearn.ensemble import RandomForestClassifier
import xgboost
from sklearn.preprocessing import normalize
from sklearn.neighbors import KNeighborsClassifier
from scipy import stats
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

In [2]:
data = pd.read_csv("TitlesTrain.csv")
data_1 = pd.read_csv("mean_text.csv")
y_train = np.array(data["target"])
groups_train = np.array(data["group"])
texts = np.array(data["information"])
texts_1 = np.array(data_1["text"])

In [4]:
data = pd.read_csv("Inf.csv")

In [6]:
data.head()

,Unnamed: 0,Unnamed: 0.1,information,group,target,doc_id
0,0,0,16 13 замена подшипник ступица ваз 21213 зам...,1,0,15731
1,1,1,ваз 2107 опт class js link ...,1,0,14829
2,2,2,ступица лада калина2 ...,1,0,15764
3,3,3,index id logo learn center классика...,1,0,17669
4,4,4,ступица нива провожать ремонт замена подши...,1,0,14852


In [215]:
# texts = []
# for i in range(data.shape[0]):
#     #print(data.iloc[i]["information"],data.iloc[i]["word"])
#     texts.append(str(data.iloc[i]["information"]) + " " + str(data.iloc[i]["word"]))

In [216]:
texts = np.concatenate((texts,texts_1))

In [217]:
p = y_train.mean()

In [218]:
for i in range(len(texts)):
    if texts[i] is np.nan:
        texts[i] = ""

In [219]:
 a = np.array([[6, 8, 3, 0],
...               [3, 2, 1, 7],
...               [8, 1, 8, 4],
...               [5, 3, 0, 5],
...               [4, 7, 5, 9]])
from scipy import stats
stats.mode(a, axis=1)[0][0]
stats.median_absolute_deviation(a, axis=1)

array([3.7065, 1.4826, 2.9652, 1.4826, 2.2239])

In [220]:
vect = TfidfVectorizer()
X_train = vect.fit_transform(texts)

In [221]:
X_train = normalize(X_train)

In [222]:

neigh = 25
XTrain = []
for i in np.unique(groups_train):
    X_ = X_train[groups_train == i]
    
    distMart = sklearn.metrics.pairwise_distances(X_, metric="cosine")
    #print(distMart)
    meanDist = np.sort(distMart, axis=1)[:,4:neigh]
    lis = []
    meanD = np.sort(distMart, axis=1)[:,:].mean(axis=1)
    stdD = np.sort(distMart, axis=1)[:,:].std(axis=1)
    mode = np.array(stats.mode(np.sort(distMart, axis=1)[:,:], axis=1))
    median = stats.median_absolute_deviation(np.sort(distMart, axis=1)[:,:], axis=1)
    #print(stdD.shape, mode.shape)
    
    DIST = sklearn.metrics.pairwise.cosine_distances(X_,  X_train[-129 - 1 + i, :])
    #print(distMart)
    #print(meanD.shape)
    for i in range(meanDist.shape[0]):
        rr = [j for j in meanDist[i,:]]
        
        rr.append(meanD[i])
        rr.append(stdD[i])
        rr.append(DIST[i])
        rr.append(median[i])
        rr.append(mode[0][i])
        rr.append(mode[1][i])
        XTrain.append(rr)
        #print(i.shape)
    
    

    
#XTrain = np.array(XTrain)
    


In [223]:
XTrain = np.array(XTrain)

In [224]:
XTrain.shape

(11690, 27)

In [225]:
with open("DBSCAN_VECTOR.txt", "r") as f:
    lis = [int(i) for i in f.read().split()]
    #print(lis)
    XTrain = np.append(XTrain, np.array(lis).reshape((-1,1)), axis=1)

In [227]:
with open("KMeans_matr.txt", "r") as f:
    lis = [int(i) for i in f.read().split()]
    lis = np.array(lis).reshape((-1,47))  
    print(lis.shape, XTrain.shape)
    XTrain = np.append(XTrain, lis, axis=1)

(11690, 47) (11690, 28)


In [228]:
# scl = StandardScaler()
# scl.fit(XTrain)
# XTrain = scl.transform(XTrain)

In [229]:
XTrain.shape, y_train.shape

((11690, 75), (11690,))

In [230]:
XTrain.shape

(11690, 75)

In [231]:
def make_big_matr(matr):
    matr1 = matr[:,:]
    for i in range(matr.shape[1]):
        for j in range(i, matr.shape[1]):
                #print((matr[:,i] * matr[:,j]).shape, matr1.shape)
                matr1 = np.concatenate((matr1, matr[:,i].reshape((-1,1)) * matr[:,j].reshape((-1,1))), axis=1)
    return matr1

In [232]:
#XTrain = make_big_matr(XTrain)

In [233]:
XTrain.shape

(11690, 75)

In [238]:
bestfeatures = SelectKBest(score_func=chi2, k=60)


In [239]:
XTrain = bestfeatures.fit_transform(XTrain,y_train)

In [240]:
folds = 8
mean_roc_auc = 0
mean_f1 = 0
for train_index, test_index in GroupKFold(n_splits=folds).split(XTrain, y_train, groups_train):
    
    xTrain = -XTrain[train_index]
    yTrain = y_train[train_index]
    
    xTest = -XTrain[test_index]
    yTest = y_train[test_index]
    clf = RandomForestClassifier(n_estimators=2000, 
                                 min_samples_leaf=2,
                                 min_samples_split=4,
                                 max_features="log2",
                                 criterion="entropy",
                                 max_depth=15,
                                 n_jobs=-1,
                                 class_weight="balanced"
                                    
                                )
    
    clf.fit(xTrain, yTrain)
    print("\there")
    res = clf.predict(xTrain)
    print(roc_auc_score(yTrain, res))
    res = clf.predict(xTest)
    print(roc_auc_score(yTest, res))
    
    a1 = clf.predict(xTest)
    threeshold = np.quantile(a1, 1-p)
    
    mean_roc_auc += roc_auc_score(yTest, a1)
    #print(a1, threeshold)
    #a1 = (a1 > threeshold).astype(int)
    print(np.unique(a1,return_counts=True))
    print("F1 ", f1_score(yTest, a1))
    mean_f1 += f1_score(yTest, a1)


print(mean_roc_auc / folds)
print(mean_f1 / folds)
    

	here
0.9621905409307244
0.8392029233987554
(array([0, 1]), array([873, 579]))
F1  0.7964285714285714
	here
0.9615791305959993
0.7861664956262522
(array([0, 1]), array([1042,  415]))
F1  0.6767295597484276
	here
0.9657397048118308
0.7932247350350798
(array([0, 1]), array([1019,  443]))
F1  0.6949352179034157
	here
0.9636724570442251
0.8140560399749547
(array([0, 1]), array([1093,  369]))
F1  0.7272727272727273
	here
0.964125071782346
0.8137458725694021
(array([0, 1]), array([1100,  375]))
F1  0.7194630872483221
	here
0.9635476426377342
0.8306883365200765
(array([0, 1]), array([984, 477]))
F1  0.7443946188340808
	here
0.9607692067615606
0.8648642043333747
(array([0, 1]), array([995, 464]))
F1  0.8070562293274532
	here
0.9663828521411272
0.7902137143762233
(array([0, 1]), array([1039,  423]))
F1  0.7035294117647058
0.8165202902292649
0.733726177940963


In [209]:
all_titles = pd.read_csv("docs_titles.tsv", sep="\t")
test_groups = pd.read_csv("test_groups.csv")
table = pd.merge(test_groups, all_titles, left_on='doc_id', right_on='doc_id',how="left")
table.shape

(16627, 4)

In [177]:
table.head()

,pair_id,group_id,doc_id,title
0,11691,130,6710,КАК ПРОПИСАТЬ АДМИНКУ В КС 1.6 СЕБЕ ИЛИ ДРУГУ ...
1,11692,130,4030,Скачать: SGL-RP доработка | Слив мода [MySQL] ...
2,11693,130,5561,Как прописать админку в кс 1.6 - Counter-Strik...
3,11694,130,4055,Как прописать простую админку в кс 1 6
4,11695,130,4247,Подбор админов для сервера по КОД_4 [Архив] -...


In [178]:
groups_test = np.array(table["group_id"])
texts = np.array(table["title"])
for i in range(len(texts)):
    if texts[i] is np.nan:
        texts[i] = ""

In [179]:
def delete_symbols(text):
    res = ""
    text = text.lower()
    
    for i in text:
        t = i    
        if not(
                (t >= '0' and t <= '9') or 
                (t >= 'a' and t <= 'z') or
                (t >= 'а' and t <= 'я')
              ):
            t = ' '
            
        res += t
        
    return res 


def list2string(myList):  
    myStr = ""  
    
    for word in myList:
        if (word not in rus_stopwords) and (word not in eng_stopwords ) and (word not in html_stopwords):
            myStr += word
    
    return myStr

def stemming(text):
    text = delete_symbols(text)
    
    tokens = mystem.lemmatize(text)
    stri = list2string(tokens)
    #print(stri)
    return stri
    

In [180]:
mystem = Mystem() 
rus_stopwords = stopwords.words("russian")
eng_stopwords = stopwords.words("english")
html_stopwords = ["span", "br", "a", "href", "img", "www", "com", "google", "ru", "html", "http", "https"]
stemming("Привет, медвед я в доме у  жен")

'привет  медведы   дом   жена\n'

In [181]:
new_texts = []
for i in texts:
    new_texts.append(stemming(i))

In [182]:
vect = TfidfVectorizer()
X_test = vect.fit_transform(new_texts)

In [183]:
pairs_id = []
groups_id = []

In [ ]:
neigh = 25
XTrain = []
for i in np.unique(groups_train):
    X_ = X_train[groups_train == i]
    
    distMart = sklearn.metrics.pairwise_distances(X_, metric="cosine")
    #print(distMart)
    meanDist = np.sort(distMart, axis=1)[:,4:neigh]
    lis = []
    meanD = np.sort(distMart, axis=1)[:,:].mean(axis=1)
    stdD = np.sort(distMart, axis=1)[:,:].std(axis=1)
    
    
    DIST = sklearn.metrics.pairwise.cosine_distances(X_,  X_train[-129 - 1 + i, :])
    #print(distMart)
    #print(meanD.shape)
    for i in range(meanDist.shape[0]):
        rr = [j for j in meanDist[i,:]]
        rr.append(meanD[i])
        rr.append(stdD[i])
        #rr.append(DIST[i])
        XTrain.append(rr)
        #print(i.shape)
    
    

    
#XTrain = np.array(XTrain)
    


In [163]:

neigh = 25
XTest = []
for i in np.unique(groups_test):
    X_ = X_test[groups_test == i]
    
    distMart = sklearn.metrics.pairwise_distances(X_, metric="cosine")
    #print(distMart)
    meanDist = np.sort(distMart, axis=1)[:,4:neigh]
    lis = []
    meanD = np.sort(distMart, axis=1)[:,:].mean(axis=1)
    stdD = np.sort(distMart, axis=1)[:,:].std(axis=1)
    for i in range(meanDist.shape[0]):
        rr = [j for j in meanDist[i,:]]
        rr.append(meanD[i])
        rr.append(stdD[i])
        rr.append(maxD[i])
        rr.append(minD[i])
        XTest.append(rr)
        #print(i.shape)
        
    

    
#XTrain = np.array(XTrain)
    


In [164]:
XTest = np.array(XTest)

In [165]:
with open("TEST_DBSCAN_VECTOR.txt", "r") as f:
    lis = [int(i) for i in f.read().split()]
    print(len(lis),XTest.shape)
    XTest = np.append(XTest, np.array(lis).reshape((-1,1)), axis=1)

16627 (16627, 25)


In [175]:
XTrain.shape, XTest.shape

((11690, 377), (16627, 377))

In [167]:
XTest = make_big_matr(XTest)

In [176]:
alpha = 0.000000000001
max_epoch = 500
C = 0.2
batch_size = 450
folds = 8
res = []
threeshold = 0.30
mean_roc_auc = 0
mean_f1 = 0
result = []
for train_index, test_index in GroupKFold(n_splits=folds).split(X_train, y_train, groups_train):
    #threeshold = 0.1
    xTrain = XTrain[train_index]
    yTrain = y_train[train_index]
    
    xTest = XTrain[test_index]
    yTest = y_train[test_index]
    
    clf = RandomForestClassifier(n_estimators=2000, 
                                 min_samples_leaf=2,
                                 min_samples_split=4,
                                 max_features="log2",
                                 criterion="entropy",
                                 max_depth=15,
                                 n_jobs=-1,
                                 class_weight="balanced"
                                    
                                )
    clf.fit(xTrain, yTrain)
    a1 = clf.predict(XTest)
    
    
    threshold = np.quantile(a1, 1-p)
    
    a1 = (a1 > threeshold).astype(int)
    result.append(a1)


print(mean_roc_auc / folds)
print(mean_f1 / folds)
    

[0 0 0 ... 1 1 1]
[0 0 0 ... 1 1 1]
[0 0 0 ... 1 1 1]
[0 0 0 ... 1 1 1]
[1 0 0 ... 1 1 1]
[0 0 0 ... 1 1 1]
[0 0 0 ... 1 1 1]
[0 0 0 ... 1 1 1]
0.0
0.0


In [182]:
r = np.array(result)
r = r.sum(axis=0)

In [183]:
kk = 2
for i in range(r.shape[0]):
    if r[i] > 5:
        r[i] = 1
    else:
        r[i] = 0

In [184]:
r = np.array(r)
r.mean(), p

(0.27671859024478257, 0.28751069289991443)

In [185]:
with open("my_submission2.csv", "w") as f:
    print("pair_id,target", file=f)
    for i in range(len(r)):
        print(table["pair_id"].iloc[i], ",", r[i], file=f, sep="")

In [127]:
np.corrcoef(XTrain[:,23],y_train)

array([[1.        , 0.57606601],
       [0.57606601, 1.        ]])

In [113]:
XTrain.shape

(11690, 24)